In [1]:
import pandahouse as ph

In [15]:
connection_test = dict(database='test',
                  host='http://clickhouse.beslan.pro:8080',
                  user='student-rw',
                  password='656e2b0c9c')

In [16]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'test',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [19]:
q = """    
    SELECT 
        purchase_date,
        purchase_id,
        client_id,
        client_age,
        client_registration_age,
        client_category,
        promotion_name,
        category_name,
        partner_name,
        client_city,
        city,
        revenue,
        quantity
    FROM
        (
        SELECT
            purchase_date,
            purchase_id,
            client_id,
            promotion_name,
            client_category,
            category_name,
            partner_name,
            (price * quantity) AS revenue,
            quantity,
            partner_id
        FROM
            (
            SELECT
                purchase_id,
                partner_id,
                client_id,
                price,
                quantity,
                purchase_date,
                status,
                CASE
                    WHEN client_category='new' THEN 'new'
                    ELSE 'old'
                END AS client_category
            FROM 
                (
                SELECT 
                    purchase_id,
                    partner_id,
                    client_id,
                    price,
                    quantity,
                    parseDateTimeBestEffort(purchase_date) AS purchase_date,
                    status
                FROM test.t_purchase
                ) AS l
            FULL JOIN
                (
                WITH (SELECT MIN(parseDateTimeBestEffort(purchase_date))
                FROM test.t_purchase) AS min_date

                SELECT 
                    client_id,
                    first_purchase,
                    CASE
                        WHEN first_purchase>=min_date THEN 'new'
                        ELSE 'old'
                    END AS client_category
                FROM
                    (
                    SELECT 
                        client_id,
                        MIN(parseDateTimeBestEffort(purchase_date)) AS first_purchase
                    FROM
                        test.t_purchase
                    GROUP BY client_id
                    )
                ) 
            AS r
                ON l.client_id=r.client_id AND l.purchase_date=r.first_purchase
                HAVING 
                    status = 'Yes' 
                    AND toDate(purchase_date) >= '2020-05-01' 
                    AND toDate(purchase_date) <= '2020-08-01'
            ) AS l
        JOIN 
            (
            SELECT
                partner_id,
                promotion_name,
                category_name,
                partner_name
            FROM test.t_promotion
            )
        AS r
            ON l.partner_id=r.partner_id
        ) AS l
    JOIN 
        (
        SELECT
            client_id,
            dateDiff('year', toDateTime(birth_date), today()) AS client_age,
            dateDiff('day', toDateTime(registration), today()) AS client_registration_age,
            client_city,
            client_city_id,
            city
        FROM test.t_clients AS l
        JOIN
            (
            SELECT
                client_city,
                client_city_id,
                city
            FROM test.t_city
            )
        AS r
            ON l.client_city_id=r.client_city_id
        )
    AS r
        ON l.client_id=r.client_id
"""

In [20]:
df = ph.read_clickhouse(query=q, connection=connection_default)

In [21]:
df

,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-05-29 22:00:00,3,35,17,-288,old,east,clothes,new14,Reading,Palmira,1719,3
1,2020-05-13 07:15:00,12,10,34,261,old,north,food,new5,Sevsk,Krishnanagar,23409,27
2,2020-05-13 18:30:00,20,45,30,220,old,west,clothes,new11,Sant'Eufemia a Maiella,Aserrí,6591,13
3,2020-07-31 23:10:00,28,32,47,-67,old,south,food,new1,Caxias,Oranienburg,12544,16
4,2020-07-20 03:36:00,32,40,15,178,old,south,food,new1,Hofstade,Göttingen,6195,7
5,2020-06-06 13:51:00,33,30,24,358,old,east,clothes,new17,Baltimore,Zapopan,8862,14
6,2020-07-18 18:04:00,37,31,43,-62,old,south,food,new1,Sacramento,Rae-Edzo,10500,20
7,2020-07-08 00:26:00,47,22,39,175,new,east,clothes,new17,Melton Mowbray,Delianuova,17982,18
8,2020-05-27 14:54:00,48,1,42,309,old,east,clothes,new14,Ivanteyevka,Fort Laird,14896,28
9,2020-06-02 07:53:00,65,7,33,-281,old,east,clothes,new13,Nashville,Toltén,5328,6
